%%capture
!pip install pymupdf4llm

In [ ]:
import pymupdf, pymupdf4llm
from pprint import pprint
import pathlib
from collections import deque

In [ ]:
input_filename = "data/bfp-a3447q.pdf"
output_filename= input_filename.split('.')[0]+'.txt'
image_path = "data/images"
margins=(50,75)
md = ""

doc = pymupdf.open(input_filename)  # use a Document for subsequent processing
my_headers = pymupdf4llm.TocHeaders(doc=doc )  # use the table of contents for determining headers
content_first_page = doc.get_toc()[0][-1] -1 # 0-based page number


In [ ]:
doc.metadata

In [ ]:
for page in doc[content_first_page : doc.page_count]:
    print(f"\rProcessing page no {page.number+1}...", end='')
    clusters = page.cluster_drawings()
    for bb in clusters:
        page.draw_rect(bb, width=0.2)  # put extra border around detected graphics
    md += pymupdf4llm.to_markdown(
        doc,
        pages=[page.number],
        margins=margins,
        hdr_info=my_headers,
        write_images=True,
        image_path = image_path,
        force_text=False
    )

In [ ]:
len(md)

In [ ]:
import regex as re
pattern = r"""[^a-zA-Z0-9!@#$%^&*()_\-+=\[\]{}|;:'",.<>/?\\`~ \t\n□△◇：±℃φ×Ω（）]"""
anti_pattern = r"""[a-zA-Z0-9!@#$%^&*()_\-+=\[\]{}|;:'",.<>/?\\`~ \t\n□△◇：±℃φ×Ω（）]"""
notes_pattern = r'Note (?=\d\))'
cleaned = re.sub(pattern, "", md)
cleaned = re.sub(notes_pattern, " *Note ", cleaned)
chaos = re.sub(anti_pattern, "", md)
splitted = re.split(r'\n(?=#)',cleaned)
splitted = [re.sub(r'^#+\s','',text) for text in splitted]
pathlib.Path("output.md").write_text(cleaned)

In [ ]:
splitted[3]

In [ ]:
#Manual correction for the missing chapters
chapter_tuples = [
    ("(1) Machine cable （Fixed type）", "(1) Machine cable（Fixed type）"),
    ("Appendix 1 ：Classification of functions using external input/output signals" , "Appendix 1 ： Classification of functions using external input/output signals")
]
chapter_name = "(1) Machine cable （Fixed type）"
chapter_toc_name="(1) Machine cable（Fixed type）"
for chapter_name, chapter_toc_name in chapter_tuples:
    chapter_index = -1
    for index, chunk in enumerate(splitted):
        if chunk.find(chapter_name) > 10: # Chapter name, if found on the beginning of string, is not a correct one
            print(f"Text: {chapter_name} found in document on index= {index} on position {chunk.find(chapter_name)}")
            chapter = chunk
            chapter_index = index
            splitted.pop(index)
            break
    if chapter_index == -1:
        print(f"Text: {chapter_name} not found in document")
    else:
        chapters = chapter.split(chapter_name,1)
        chapters[-1] = chapter_toc_name + chapters[-1]
        for chapter in chapters[::-1]:
            splitted.insert(chapter_index,chapter)
        
    

In [ ]:
pathlib.Path("data/chaos.md").write_bytes(chaos.encode())

In [ ]:
len(splitted)

In [ ]:
chunk_dict = { chunk.split('\n',1)[0] : chunk for chunk in splitted}

In [ ]:
toc = doc.get_toc()

In [ ]:
retries = []
for chapter in toc:
    title = chapter[1]
    result = list(filter(lambda i: title in i[0], chunk_dict.items()))
    if len(result) == 0:
        print(f"Found {len(result)} matches for {title}")
        
    elif len(result) > 1:
        retries.append(chapter)
        print(f"Found {len(result)} matches for {title}")
    else:
        content = result[0][-1]
        chapter.append(content)
        chunk_dict.pop(result[0][0])

for chapter in retries:
    title = chapter[1]
    result = list(filter(lambda i: title in i[0], chunk_dict.items()))
    if len(result) == 0:
        print(f"Found {len(result)} matches for {title}")
        
    elif len(result) > 1:
        retries.append(title)
        print(f"Found {len(result)} matches for {title}")
    else:
        print(f"Found finally {len(result)} matches for {title}")
        content = result[0][-1]
        chapter.append(content)
        chunk_dict.pop(result[0][0])
assert len(chunk_dict) == 0